# Deeper Dive on Process Analysis

### Users
To analyze users associated with processes you can use the plugin `getsids`, SIDs standing for security identifier. You should see SIDs for local and domain users

In [5]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 getsids

Volatility Foundation Volatility Framework 2.6
System (4): S-1-5-18 (Local System)
System (4): S-1-5-32-544 (Administrators)
System (4): S-1-1-0 (Everyone)
System (4): S-1-5-11 (Authenticated Users)
smss.exe (564): S-1-5-18 (Local System)
smss.exe (564): S-1-5-32-544 (Administrators)
smss.exe (564): S-1-1-0 (Everyone)
smss.exe (564): S-1-5-11 (Authenticated Users)
csrss.exe (636): S-1-5-18 (Local System)
csrss.exe (636): S-1-5-32-544 (Administrators)
csrss.exe (636): S-1-1-0 (Everyone)
csrss.exe (636): S-1-5-11 (Authenticated Users)
winlogon.exe (660): S-1-5-18 (Local System)
winlogon.exe (660): S-1-5-32-544 (Administrators)
winlogon.exe (660): S-1-1-0 (Everyone)
winlogon.exe (660): S-1-5-11 (Authenticated Users)
services.exe (704): S-1-5-18 (Local System)
services.exe (704): S-1-5-32-544 (Administrators)
services.exe (704): S-1-1-0 (Everyone)
services.exe (704): S-1-5-11 (Authenticated Users)
lsass.exe (716): S-1-5-18 (Local System)
lsass.exe (716): S-1-5-32-544 (Administrators)
lsass

Reviewing all the SIDs can be daunting so try to focus just on the process we identified as supicoius and parent processes in this case PID (796)

### Privilleges

You can also see what privilleges are associated with each user if you do not have access to that infomration in other formats (like if you don't have access to the affected domain). You can enumerate privilleges with the privs command as shown below:

In [7]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 privs

Volatility Foundation Volatility Framework 2.6
Pid      Process          Value  Privilege                            Attributes               Description
-------- ---------------- ------ ------------------------------------ ------------------------ -----------
       4 System                7 SeTcbPrivilege                       Present,Enabled,Default  Act as part of the operating system
       4 System                2 SeCreateTokenPrivilege               Present                  Create a token object
       4 System                9 SeTakeOwnershipPrivilege             Present                  Take ownership of files/objects
       4 System               15 SeCreatePagefilePrivilege            Present,Enabled,Default  Create a pagefile
       4 System                4 SeLockMemoryPrivilege                Present,Enabled,Default  Lock pages in memory
       4 System                3 SeAssignPrimaryTokenPrivilege        Present                  Replace a process-level token
       4 S

### PE Version Information

In [9]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 verinfo

Volatility Foundation Volatility Framework 2.6
\SystemRoot\System32\smss.exe
C:\WINDOWS\system32\ntdll.dll
\??\C:\WINDOWS\system32\csrss.exe
C:\WINDOWS\system32\ntdll.dll
C:\WINDOWS\system32\CSRSRV.dll
C:\WINDOWS\system32\basesrv.dll
C:\WINDOWS\system32\winsrv.dll
  File version    : 5.1.2600.5512
  Product version : 5.1.2600.5512
  Flags           : 
  OS              : Windows NT
  File Type       : Dynamic Link Library
  File Date       : 
  CompanyName : Microsoft Corporation 
  FileDescription : Windows Server DLL 
  FileVersion : 5.1.2600.5512 (xpsp.080413-2105) 
  InternalName : winsrv 
  LegalCopyright : \xa9 Microsoft Corporation. All rights reserved. 
  OriginalFilename : winsrv.dll 
  ProductName : Microsoft\xae Windows\xae Operating System 
  ProductVersion : 5.1.2600.5512 
C:\WINDOWS\system32\GDI32.dll
C:\WINDOWS\system32\KERNEL32.dll
C:\WINDOWS\system32\USER32.dll
  File version    : 5.1.2600.5512
  Product version : 5.1.2600.5512
  Flags           : 
  OS              : 

### Loaded DLL's

In [11]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 dlllist --pid=796

******************************************************************
iexplore.exe pid:    796
Command line : "C:\Program Files\Internet Explorer\iexplore.exe"
Service Pack 3

Base             Size  LoadCount LoadTime                       Path
---------- ---------- ---------- ------------------------------ ----
0x00400000    0x9b000     0xffff                                C:\Program Files\Internet Explorer\iexplore.exe
0x7c900000    0xaf000     0xffff                                C:\WINDOWS\system32\ntdll.dll
0x7c800000    0xf6000     0xffff                                C:\WINDOWS\system32\kernel32.dll
0x77dd0000    0x9b000     0xffff                                C:\WINDOWS\system32\ADVAPI32.dll
0x77e70000    0x92000     0xffff                                C:\WINDOWS\system32\RPCRT4.dll
0x77fe0000    0x11000     0xffff                                C:\WINDOWS\system32\Secur32.dll
0x77f10000    0x49000     0xffff                                C:\WINDOWS\system32\GDI32.dll
0x7e

In [13]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 handles --pid=796

Volatility Foundation Volatility Framework 2.6
Offset(V)     Pid     Handle     Access Type             Details
---------- ------ ---------- ---------- ---------------- -------
0x821ace98    796        0xc   0x100020 File             \Device\HarddiskVolume1\WINDOWS\system32
0x82189028    796      0x578   0x12019f File             \Device\NamedPipe\ROUTER
0x822eff10    796      0x57c   0x1f0003 Event            
0x81f229d0    796      0x580   0x100003 Semaphore        
0x81f22a08    796      0x584   0x100003 Semaphore        
0x81f30838    796      0x588   0x100003 Semaphore        
0x822eae40    796      0x58c   0x100003 Semaphore        
0xe1c29c90    796      0x590    0x20019 Key              MACHINE\SYSTEM\CONTROLSET001\SERVICES\NETBT\PARAMETERS
0xe1117fb8    796      0x594    0x20019 Key              MACHINE\SYSTEM\CONTROLSET001\SERVICES\NETBT\PARAMETERS\INTERFACES
0xe1c29c28    796      0x598    0x20019 Key              MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\PARAMETERS
0xe1e9

You can also limit what handles are enumerated by using the `-t ObjectType` as shown below:

In [19]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 handles --pid=796 -t File,Key

Volatility Foundation Volatility Framework 2.6
Offset(V)     Pid     Handle     Access Type             Details
---------- ------ ---------- ---------- ---------------- -------
0x821ace98    796        0xc   0x100020 File             \Device\HarddiskVolume1\WINDOWS\system32
0x82189028    796      0x578   0x12019f File             \Device\NamedPipe\ROUTER
0xe1c29c90    796      0x590    0x20019 Key              MACHINE\SYSTEM\CONTROLSET001\SERVICES\NETBT\PARAMETERS
0xe1117fb8    796      0x594    0x20019 Key              MACHINE\SYSTEM\CONTROLSET001\SERVICES\NETBT\PARAMETERS\INTERFACES
0xe1c29c28    796      0x598    0x20019 Key              MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\PARAMETERS
0xe1e9f728    796      0x59c    0x20019 Key              MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\LINKAGE
0x81db29b0    796      0x5a8   0x1200a0 File             \Device\Ip
0x81db2a48    796      0x5ac   0x100003 File             \Device\Ip
0x81f0f7f0    796      0x5b0   0x1200a0 File       

## Command Line History
### cmdscan
You can see the command line history by enumerating COMMAND_HISTORY objects from memory. There are different plugins that can parse these data points. This enumeration works by looking for a known constant called `(MaxHistory)` which is a parameter that stores the max number of commands to keep in a buffer. These structures are stored in `csrss.exe` in XP/2003/Vista/2008 and `conhost.exe` in Windows 7. The plugin to review this data on XP systems is below.

In [23]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 cmdscan

Volatility Foundation Volatility Framework 2.6
**************************************************
CommandProcess: csrss.exe Pid: 636
CommandHistory: 0x1007910 Application: MIRAgent.exe Flags: Allocated
CommandCount: 0 LastAdded: -1 LastDisplayed: -1
FirstCommand: 0 CommandCountMax: 50
ProcessHandle: 0x104
**************************************************
CommandProcess: csrss.exe Pid: 636
CommandHistory: 0x10132d8 Application: cmd.exe Flags: Allocated, Reset
CommandCount: 2 LastAdded: 1 LastDisplayed: 1
FirstCommand: 0 CommandCountMax: 50
ProcessHandle: 0x308
Cmd #0 @ 0x1013160: cd "c:\Program Files\Mandiant\Mandiant Intelligent Response Agent"
Cmd #1 @ 0x1007790: MIRAgent.exe -o -script MalwareAudit.Batch.xml -encoding none
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 cmdscan


### consoles
You can also find more command line parameter data by parsing the `CONSOLE_INFORMATION` objects in memory. These objects not only show the command being executed but also the output buffer from the command. This command can be useful espeically if you are looking for attackers exiltrating data, users and passwords being used, archive file passwords, or if their malware has command parameters which are required to be provided when ran. The plugin to view this data is below:

In [25]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 consoles

Volatility Foundation Volatility Framework 2.6
**************************************************
ConsoleProcess: csrss.exe Pid: 636
Console: 0x4f2378 CommandHistorySize: 50
HistoryBufferCount: 2 HistoryBufferMax: 4
OriginalTitle: %SystemRoot%\system32\cmd.exe
Title: C:\WINDOWS\system32\cmd.exe - MIRAgent.exe -o -script MalwareAudit.Batch.xml -encoding none
AttachedProcess: MIRAgent.exe Pid: 456 Handle: 0x104
AttachedProcess: cmd.exe Pid: 840 Handle: 0x308
----
CommandHistory: 0x1007910 Application: MIRAgent.exe Flags: Allocated
CommandCount: 0 LastAdded: -1 LastDisplayed: -1
FirstCommand: 0 CommandCountMax: 50
ProcessHandle: 0x104
----
CommandHistory: 0x10132d8 Application: cmd.exe Flags: Allocated, Reset
CommandCount: 2 LastAdded: 1 LastDisplayed: 1
FirstCommand: 0 CommandCountMax: 50
ProcessHandle: 0x308
Cmd #0 at 0x1013160: cd "c:\Program Files\Mandiant\Mandiant Intelligent Response Agent"
Cmd #1 at 0x1007790: MIRAgent.exe -o -script MalwareAudit.Batch.xml -encoding none
----
Scree